In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd

In [3]:
from utils.print_utils import print_json_tree

In [13]:
# load the data at ../data/Dataset.json
with open('../data/Dataset.json') as f:
    data = json.load(f)

# print tree of first 2 levels
print(f'Data tree for first 2 levels:')
print_json_tree(data, max_depth=2)

Data tree for first 2 levels:
- status
- message
  - version
  - utc_age
  - content


In [16]:
dataset = pd.DataFrame(data['message']['content'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   doc_type                    285 non-null    object
 1   criticality_tag             285 non-null    object
 2   knowledge_type              285 non-null    object
 3   requirement_list            285 non-null    object
 4   title_list                  285 non-null    object
 5   location_list               285 non-null    object
 6   description_list            281 non-null    object
 7   internal_rating_list        284 non-null    object
 8   internal_ratingsource_list  285 non-null    object
 9   cvss_rating_list            229 non-null    object
 10  rule_list                   285 non-null    object
 11  cwe_id_list                 195 non-null    object
 12  cve_id_list                 253 non-null    object
 13  activity_list               285 non-null    object